In [73]:
from langfuse import get_client
from dotenv import load_dotenv
import pprint
load_dotenv()

langfuse = get_client()

In [96]:
# x = langfuse.api.trace.get("3c7300a2c603e23e4cd87f07cdc5c34f")
# x = langfuse.api.trace.get("6f338e14efc427b291ecb6fce691bfae")
# x = langfuse.api.observations.get_many(trace_id="18899600d47d490b1fb7026522dac032")
# x = langfuse.api.observations.get_many(trace_id="701786ffa391011e9b1d4248654ab632")
# x = langfuse.api.observations.get_many(trace_id="c53266426544709153cc11f0ea670c93")
# x = langfuse.api.observations.get_many(trace_id="618435328066ae7cdb2d9155377c70bd")
# x = langfuse.api.observations.get_many(trace_id="e64a391779b5b9fcb052c0bce5ec2377")
# x = langfuse.api.observations.get_many(trace_id="13ac9c71bfd2d9b4d78b0ccdb33768b0")
# x = langfuse.api.observations.get_many(trace_id="785647f004712c1dc6db1e3cbeb4d95a") #see this 
# x = langfuse.api.observations.get_many(trace_id="5608b24c0795cab5d7db259faa0f5e26")
# x = langfuse.api.observations.get_many(trace_id="3117e6be8c667dcf3b2c23db7e18f922") 
x = langfuse.api.observations.get_many(trace_id="ac1fb98ec344cb89da2cd137032e15a3") 



In [97]:
y=x.dict()
y

{'data': [{'id': 'aca400e9226e6955',
   'traceId': 'ac1fb98ec344cb89da2cd137032e15a3',
   'type': 'GENERATION',
   'name': 'call_llm',
   'startTime': datetime.datetime(2025, 12, 1, 9, 20, 25, 13000, tzinfo=datetime.timezone.utc),
   'endTime': datetime.datetime(2025, 12, 1, 9, 20, 34, 513000, tzinfo=datetime.timezone.utc),
   'completionStartTime': None,
   'model': 'azure/gpt-4o-mini',
   'modelParameters': {},
   'input': {'model': 'azure/gpt-4o-mini',
    'config': {'system_instruction': '\n        **Role:** Host agent for the agent-to-agent protocol; delegates queries to specialized remote agents with maximum efficiency.\n\n**Core Directives:**\n\n* **Task Delegation:** Use the `send_message` function to assign precise, actionable tasks to remote agents.\n* **Full Context Provision:** If an agent repeatedly asks for user confirmation, it likely lacks conversation history. Include all relevant context in the task to prevent this.\n* **Autonomous Multi-Agent Engagement:** Engage any

In [98]:
# logger = setup_logger()
# import time
# def main():
    # start_time=time.time()
    # logger.info(f"Started processing :{start_time}")
steps = []
total_responses=[]
for obs in y['data']:
    if obs['type'] == "GENERATION":
            contents = obs['input']['contents']
            for entry in contents:
                for num, part in enumerate(entry["parts"]):
                    if "function_call" in part:
                        fc = part["function_call"]
                        pending_call = {
                        "name": fc["args"]["agent_name"],
                        "task": fc["args"]["task"],
                        "output": None,
                        "index": num,
                        "artifactId":False
                        }
                        total_responses.append(pending_call)
                    
                    if "function_response" in part:

                        for getdict in total_responses:
                            if getdict["index"]==num and getdict["artifactId"]==False:
                                dict_to_update=getdict

                                contains_result=part["function_response"].get("response")
                                if contains_result.get("result"):
                                    inter_res=contains_result["result"]["artifacts"]
                                elif contains_result.get("task"):
                                    inter_res=contains_result["task"]["artifacts"]
                               
                                if inter_res:
                                    if len(inter_res)==1:
                                        intermeditate_result= inter_res[0]["parts"][0]
                                        if intermeditate_result["kind"]=="text":
                                            # print(intermeditate_result["text"])
                                            dict_to_update["output"]=intermeditate_result["text"]
                                            dict_to_update["artifactId"]=True


                                        else:
                                            # logger.info("Logic falied . Did not find expected text find for output")
                                            raise "Logic falied . Did not find expected text find for output"
                                else:
                                    # logger.info("Did not find the function response structure..")
                                    raise "Did not find the function response structure"
                         

In [99]:
total_responses

[{'name': 'Travel Planner Agent',
  'task': 'Plan a 7-day trip to Japan from Bangalore for 2 people in October. Include an itinerary with flights, accommodation, activities, and local transportation.',
  'output': '{\n  "trip_summary": "A week-long adventure in Japan, blending cultural experiences, culinary delights, and stunning landscapes, with a focus on Tokyo and Kyoto.",\n  "duration": "7 days / 6 nights",\n  "itinerary": [\n    {\n      "day": 1,\n      "date": "2023-10-01",\n      "title": "Arrival in Tokyo",\n      "schedule": [\n        "Morning: Depart from Kempegowda International Airport, Bangalore.",\n        "Afternoon: Arrive at Narita International Airport, Tokyo. Take the Narita Express to Tokyo Station.",\n        "Evening: Check into your hotel in Shinjuku area. Dinner at a local Izakaya."\n      ],\n      "accommodation": "Hotel Gracery Shinjuku",\n      "notes": "Try to get a view of the Godzilla head on the hotel."\n    },\n    {\n      "day": 2,\n      "date": "2

In [100]:
# logger = setup_logger()
# import time
# def main():
    # start_time=time.time()
    # logger.info(f"Started processing :{start_time}")
steps = []
total_responses=[]
for obs in y['data']:
    if obs['type'] == "GENERATION":
            contents = obs['input']['contents']
            for entry in contents:
                for num, part in enumerate(entry["parts"]):
                    if "function_call" in part:
                        fc = part["function_call"]
                        pending_call = {
                        "name": fc["args"]["agent_name"],
                        "task": fc["args"]["task"],
                        "output": None,
                        "index": num,
                        "artifactId":False
                        }
                        total_responses.append(pending_call)
                    
                    if "function_response" in part:

                        for getdict in total_responses:
                            if getdict["index"]==num and getdict["artifactId"]==False:
                                dict_to_update=getdict
                                #   print(dict_to_update)
                                # contains_result=part["function_response"].get("response")["result"]["artifacts"]
                                contains_result=part["function_response"].get("response")["task"]["artifacts"]
                                if part["function_response"].get("response")["result"]:
                                    
                                # result_final=part["function_response"].get("response")
                                # if result_final["result"]:
                                #     contains_result = ["result"]["artifacts"]
                                # else:
                                #     contains_result = ["task"]["artifacts"]
                                pprint.pprint(contains_result)
                                print(len(contains_result))
                                if contains_result:
                                    if len(contains_result)==1:
                                        intermeditate_result= contains_result[0]["parts"][0]
                                        if intermeditate_result["kind"]=="text":
                                            # print(intermeditate_result["text"])
                                            dict_to_update["output"]=intermeditate_result["text"]
                                            dict_to_update["artifactId"]=True


                                        else:
                                            # logger.info("Logic falied . Did not find expected text find for output")
                                            raise "Logic falied . Did not find expected text find for output"
                                else:
                                    # logger.info("Did not find the function response structure..")
                                    raise "Did not find the function response structure"
                         

IndentationError: expected an indented block after 'if' statement on line 32 (2076853489.py, line 39)

In [72]:
total_responses

[{'name': 'Travel Planner Agent',
  'task': 'User wants to plan a journey to the Netherlands. Please create a detailed trip itinerary that includes travel arrangements, accommodation suggestions, and activities or attractions to visit.',
  'output': '{\n  "trip_summary": "Explore the beautiful canals, rich history, and vibrant culture of the Netherlands on this 7-day adventure.",\n  "duration": "7 days / 6 nights",\n  "itinerary": [\n    {\n      "day": 1,\n      "date": "YYYY-MM-DD (if known)",\n      "title": "Arrival in Amsterdam",\n      "schedule": [\n        "Morning: Arrive at Amsterdam Schiphol Airport. Take a train to Amsterdam Central Station (20 min).",\n        "Afternoon: Check into your hotel in the city center. Explore the nearby Dam Square and visit the Royal Palace.",\n        "Evening: Dinner at a local restaurant, try traditional Dutch pancakes."\n      ],\n      "accommodation": "Hotel Estherea or similar (canal-side hotel)",\n      "notes": "Purchase an I Amsterdam

In [39]:
input = None
output = None
for obs in y['data']:
        if obs['type'] == "GENERATION":
                input = obs['input']['contents'][0]['parts'][0]['text']
                contents = obs['output']['content']
                for sub_content in contents['parts']:
                        if sub_content.get('text'):
                                output = sub_content.get('text')
                trace_id = obs['traceId']
                startTime = obs['startTime']
                model = obs['model']
                level = y['meta']['totalItems']
                environment = obs['environment']
                input_tokens = obs['usageDetails']['input']
                output_tokens = obs['usageDetails']['output']
                total_tokens = obs['usageDetails']['total']
                latency = obs['latency']

        if output== None:
                Agents = [obs for obs in y['data'] if obs.get('type') == 'AGENT']
                latest_agent = min(Agents, key=lambda x: x['endTime'])
                # output = latest_agent['output']['result']['artifacts'][0]['parts']['text']
                artifacts = latest_agent['output']['result']['artifacts']
                first_artifact = artifacts[0]
                parts = first_artifact['parts']
                output = parts[0]['text']


In [33]:
y

{'data': [{'id': 'bb83a3a7ef6133c0',
   'traceId': '6d2e4f8cd69b483dabaa7863be2b57e0',
   'type': 'GENERATION',
   'name': 'call_llm',
   'startTime': datetime.datetime(2025, 11, 27, 7, 38, 8, 361000, tzinfo=datetime.timezone.utc),
   'endTime': datetime.datetime(2025, 11, 27, 7, 38, 15, 282000, tzinfo=datetime.timezone.utc),
   'completionStartTime': None,
   'model': 'azure/gpt-4o-mini',
   'modelParameters': {},
   'input': {'model': 'azure/gpt-4o-mini',
    'config': {'system_instruction': '\n        **Role:** Host agent for the agent-to-agent protocol; delegates queries to specialized remote agents with maximum efficiency.\n\n**Core Directives:**\n\n* **Task Delegation:** Use the `send_message` function to assign precise, actionable tasks to remote agents.\n* **Full Context Provision:** If an agent repeatedly asks for user confirmation, it likely lacks conversation history. Include all relevant context in the task to prevent this.\n* **Autonomous Multi-Agent Engagement:** Engage a

In [40]:
print(input)

summarize agentic framework for me 
